In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Rondônia - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [5]:
data = pd.read_csv('2003_01_model_input_RO.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Rondônia - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rondônia - Produção de Cimento (t),Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - Desemprego,Rondônia - Consumo de Cimento (t)
0,2003,0.688843,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,37.228190,...,21.936746,80.587098,1.323882,59.844000,2.068350e+07,1.587672e+06,11.468162,1.828350e+07,8.226062,185.386000
1,2004,0.691090,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,36.568145,...,21.867756,81.021529,1.419000,59.844000,2.100894e+07,1.609824e+06,11.519650,1.840022e+07,8.152626,209.004000
2,2005,0.693216,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,35.723999,...,21.784775,81.685569,1.553394,59.844000,2.130613e+07,1.615999e+06,11.569863,1.848933e+07,8.130621,233.057000
3,2006,0.695228,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,35.295451,...,21.770513,82.191603,1.640246,59.844000,2.157640e+07,1.633992e+06,11.602119,1.855816e+07,8.038397,272.001000
4,2007,0.697131,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,35.882176,...,21.907679,82.152029,1.592738,64.041487,2.184313e+07,1.630117e+06,11.647841,1.864093e+07,8.013839,351.377000
5,2008,0.698932,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,36.645621,...,22.070496,80.670838,1.321447,150.448962,2.208447e+07,1.637885e+06,11.689351,1.870941e+07,7.979888,390.788000
6,2009,0.700638,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,33.687787,...,21.933676,79.031178,1.250582,336.177608,2.226473e+07,1.643078e+06,11.684640,1.873529e+07,8.008176,452.325000
7,2010,0.702254,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,31.961436,...,21.812895,78.858536,1.335344,502.201514,2.285838e+07,1.640521e+06,11.724602,1.891099e+07,7.971248,921.425000
8,2011,0.703786,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,31.660273,...,21.892035,79.749795,1.413919,583.009861,2.338128e+07,1.618454e+06,11.764179,1.903017e+07,7.956614,1021.570000
9,2012,0.705401,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,31.707927,...,21.924985,80.004781,1.485512,560.443805,2.383098e+07,1.560372e+06,11.757682,1.910530e+07,6.546489,933.501000


In [6]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Rondônia - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rondônia - Produção de Cimento (t),Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - Desemprego
0,0.688843,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,37.228190,1012.461097,...,25.895984,21.936746,80.587098,1.323882,59.844000,2.068350e+07,1.587672e+06,11.468162,1.828350e+07,8.226062
1,0.691090,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,36.568145,1010.368502,...,25.743140,21.867756,81.021529,1.419000,59.844000,2.100894e+07,1.609824e+06,11.519650,1.840022e+07,8.152626
2,0.693216,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,35.723999,1007.144098,...,25.533680,21.784775,81.685569,1.553394,59.844000,2.130613e+07,1.615999e+06,11.569863,1.848933e+07,8.130621
3,0.695228,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,35.295451,1003.804270,...,25.431108,21.770513,82.191603,1.640246,59.844000,2.157640e+07,1.633992e+06,11.602119,1.855816e+07,8.038397
4,0.697131,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,35.882176,1001.365408,...,25.598931,21.907679,82.152029,1.592738,64.041487,2.184313e+07,1.630117e+06,11.647841,1.864093e+07,8.013839
5,0.698932,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,36.645621,1000.662849,...,26.132961,22.070496,80.670838,1.321447,150.448962,2.208447e+07,1.637885e+06,11.689351,1.870941e+07,7.979888
6,0.700638,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,33.687787,1000.539325,...,26.389508,21.933676,79.031178,1.250582,336.177608,2.226473e+07,1.643078e+06,11.684640,1.873529e+07,8.008176
7,0.702254,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,31.961436,1000.552176,...,26.284966,21.812895,78.858536,1.335344,502.201514,2.285838e+07,1.640521e+06,11.724602,1.891099e+07,7.971248
8,0.703786,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,31.660273,998.679132,...,26.126469,21.892035,79.749795,1.413919,583.009861,2.338128e+07,1.618454e+06,11.764179,1.903017e+07,7.956614
9,0.705401,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,31.707927,990.533026,...,26.091282,21.924985,80.004781,1.485512,560.443805,2.383098e+07,1.560372e+06,11.757682,1.910530e+07,6.546489


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      209.004000
1      233.057000
2      272.001000
3      351.377000
4      390.788000
5      452.325000
6      921.425000
7     1021.570000
8      933.501000
9      826.670000
10     614.394714
11     510.273771
12     410.580514
13     406.814000
14     376.138000
15     399.441000
16     488.194000
17     502.824000
18     501.142000
19            NaN
Name: Rondônia - Consumo de Cimento (t), dtype: float64

In [8]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Rondônia - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rondônia - Produção de Cimento (t),Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - Desemprego
0,-1.768808,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,1.680683,1.940071,...,-0.710591,0.160959,0.602716,-0.538784,-1.149207,-1.622684,0.384181,-2.216191,-1.841163,1.166353
1,-1.495156,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,1.352220,1.643488,...,-1.095142,-0.243927,0.869044,0.183661,-1.149207,-1.414824,0.683585,-1.737423,-1.507080,1.018944
2,-1.236203,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,0.932141,1.186495,...,-1.622140,-0.730922,1.276132,1.204416,-1.149207,-1.225013,0.767051,-1.270509,-1.252031,0.974772
3,-0.991191,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,0.718879,0.713143,...,-1.880208,-0.814626,1.586355,1.864078,-1.149207,-1.052390,1.010242,-0.970566,-1.055042,0.789652
4,-0.759364,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,1.010855,0.367484,...,-1.457970,-0.009629,1.562094,1.503240,-1.125291,-0.882035,0.957865,-0.545420,-0.818130,0.740357
5,-0.539966,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,1.390774,0.267911,...,-0.114363,0.945916,0.654053,-0.557281,-0.632962,-0.727890,1.062859,-0.159427,-0.622119,0.672206
6,-0.332239,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,-0.081155,0.250404,...,0.531101,0.142946,-0.351137,-1.095518,0.425273,-0.612763,1.133050,-0.203230,-0.548059,0.728989
7,-0.135428,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,-0.940253,0.252225,...,0.268076,-0.565892,-0.456975,-0.451731,1.371236,-0.233603,1.098488,0.168358,-0.045155,0.654863
8,0.051225,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,-1.090123,-0.013241,...,-0.130699,-0.101440,0.089409,0.145069,1.831662,0.100371,0.800233,0.536376,0.295942,0.625489
9,0.247948,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,-1.066409,-1.167784,...,-0.219227,0.091942,0.245728,0.688831,1.703087,0.387593,0.015197,0.475959,0.510983,-2.205064


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      209.004000
1      233.057000
2      272.001000
3      351.377000
4      390.788000
5      452.325000
6      921.425000
7     1021.570000
8      933.501000
9      826.670000
10     614.394714
11     510.273771
12     410.580514
13     406.814000
14     376.138000
15     399.441000
16     488.194000
Name: Rondônia - Consumo de Cimento (t), dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Rondônia - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rondônia - Produção de Cimento (t),Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - Desemprego
17,0.923679,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,0.635756,-0.011926,...,0.437313,-0.638697,-0.57391,-0.346953,-0.082464,0.874068,-0.785213,0.698552,0.753401,-1.482458


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    502.824
Name: Rondônia - Consumo de Cimento (t), dtype: float64

In [12]:
train_input.iloc[-(len(train_input)//5):]

,Rondônia - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rondônia - Produção de Cimento (t),Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - Desemprego
14,1.405167,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,0.294499,-0.356994,...,1.168358,-0.934434,-1.498503,-0.206912,-0.197348,1.243290,-1.413734,1.196729,1.193375,-0.728473
15,1.337642,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,-0.519246,-0.102571,...,0.862643,-1.276902,-1.461385,-1.418404,-0.257552,1.198503,-1.288925,1.145168,1.080670,-0.989258
16,1.164322,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,-1.867606,-0.060208,...,0.703906,-1.580362,-1.505805,-2.141832,-0.161984,1.099624,-1.061532,0.950531,0.962531,-1.286188


In [13]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [16]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [17]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1759688990, 1643825328, 3111390964, 37587328, 1338623368, 629497497, 4114234670, 348993400, 327625817, 1903168762, 1431955707, 2623254933, 1332629167, 621255697, 3741040231, 333166774, 2631454135, 2352572479, 4199393571, 2884987240, 3001573848, 2803612978, 860519479, 3947704560, 1111035839, 763778555, 1586065412, 2659764529, 1845890156, 1605102739, 61818555, 1646151125, 4000709116, 3583886679, 2063047802, 2177280861, 610496141, 74561904, 1184446424, 2295867771, 3497014859, 3239228146, 3866582755, 1190558790, 128545380, 245174916, 862893515, 3680415014, 2399178780, 527237727, 987863429, 505415286, 1662456110, 1168844543, 2408127237, 2271348618, 1826829221, 1419046760, 3083212168, 1929060935, 3965483823, 1545454854, 3036454595, 1959309105, 2753099210, 63606053, 2721637616, 3084485192, 1373039215, 2717091470, 3255431788, 4261589813, 2495714768, 3227929372, 1356812722, 4172008437, 3388610210, 692044800, 983787170, 195241024, 948624114, 2337309750, 1951954690, 3287004011, 3814556507, 86457

val_loss: 43686.40625


Step: 87 ___________________________________________
val_loss: 43735.99609375


Step: 88 ___________________________________________
val_loss: 5296.82373046875


Step: 89 ___________________________________________
val_loss: 43725.41796875


Step: 90 ___________________________________________
val_loss: 5475.22216796875


Step: 91 ___________________________________________
val_loss: 4502.09326171875


Step: 92 ___________________________________________
val_loss: 2360.0244140625


Step: 93 ___________________________________________
val_loss: 43704.08203125


Step: 94 ___________________________________________
val_loss: 43734.578125


Step: 95 ___________________________________________
val_loss: 7889.54638671875


Step: 96 ___________________________________________
val_loss: 3513.117919921875


Step: 97 ___________________________________________
val_loss: 43273.94140625


Step: 98 ___________________________________________
val_loss: 43735.22265625


Step: 

2023-10-05 15:02:27.260307: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 2681.366455078125


final_seed: 527237727


In [18]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 492ms/step - loss: 330231.8125 - val_loss: 172085.9219
Epoch 2/10000
1/1 [==============================] - 0s 15ms/step - loss: 324165.8438 - val_loss: 225067.5000
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 284025.7500 - val_loss: 10672.6104
Epoch 4/10000
1/1 [==============================] - 0s 18ms/step - loss: 222561.3906 - val_loss: 6860.6294
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 228925.4844 - val_loss: 18799.4707
Epoch 6/10000
1/1 [==============================] - 0s 15ms/step - loss: 224340.8906 - val_loss: 12669.0127
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 220664.8125 - val_loss: 13458.4180
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 217276.0625 - val_loss: 28104.2031
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 178016.3125 - val_loss: 417163.3438
Epoch 10/10000
1

1/1 [==============================] - 0s 16ms/step - loss: 38586.2656 - val_loss: 4460.3726
Epoch 77/10000
1/1 [==============================] - 0s 16ms/step - loss: 41304.5234 - val_loss: 7564.1187
Epoch 78/10000
1/1 [==============================] - 0s 16ms/step - loss: 40654.4961 - val_loss: 7245.6523
Epoch 79/10000
1/1 [==============================] - 0s 15ms/step - loss: 40039.7734 - val_loss: 6414.0527
Epoch 80/10000
1/1 [==============================] - 0s 15ms/step - loss: 38690.0938 - val_loss: 9867.9854
Epoch 81/10000
1/1 [==============================] - 0s 14ms/step - loss: 38874.8477 - val_loss: 5921.1099
Epoch 82/10000
1/1 [==============================] - 0s 14ms/step - loss: 38774.0586 - val_loss: 5842.3354
Epoch 83/10000
1/1 [==============================] - 0s 14ms/step - loss: 37923.9727 - val_loss: 6275.3359
Epoch 84/10000
1/1 [==============================] - 0s 15ms/step - loss: 37144.8477 - val_loss: 5959.3452
Epoch 85/10000
1/1 [=======================

1/1 [==============================] - 0s 16ms/step - loss: 20867.3379 - val_loss: 2404.7373
Epoch 152/10000
1/1 [==============================] - 0s 16ms/step - loss: 20304.8984 - val_loss: 2381.0588
Epoch 153/10000
1/1 [==============================] - 0s 16ms/step - loss: 20216.5215 - val_loss: 2391.6484
Epoch 154/10000
1/1 [==============================] - 0s 16ms/step - loss: 20065.9844 - val_loss: 2377.3701
Epoch 155/10000
1/1 [==============================] - 0s 15ms/step - loss: 19964.7402 - val_loss: 2391.6101
Epoch 156/10000
1/1 [==============================] - 0s 16ms/step - loss: 19645.9824 - val_loss: 2331.0027
Epoch 157/10000
1/1 [==============================] - 0s 16ms/step - loss: 19805.2559 - val_loss: 14057.7930
Epoch 158/10000
1/1 [==============================] - 0s 16ms/step - loss: 28924.4609 - val_loss: 2375.2593
Epoch 159/10000
1/1 [==============================] - 0s 15ms/step - loss: 21046.6113 - val_loss: 2586.3079
Epoch 160/10000
1/1 [=============

1/1 [==============================] - 0s 14ms/step - loss: 17465.9570 - val_loss: 2561.7832
Epoch 227/10000
1/1 [==============================] - 0s 14ms/step - loss: 17395.0332 - val_loss: 2647.0286
Epoch 228/10000
1/1 [==============================] - 0s 14ms/step - loss: 17370.3281 - val_loss: 2570.3735
Epoch 229/10000
1/1 [==============================] - 0s 14ms/step - loss: 17221.7188 - val_loss: 2564.4358
Epoch 230/10000
1/1 [==============================] - 0s 14ms/step - loss: 17422.7227 - val_loss: 2495.2031
Epoch 231/10000
1/1 [==============================] - 0s 14ms/step - loss: 17050.2539 - val_loss: 2716.5801
Epoch 232/10000
1/1 [==============================] - 0s 14ms/step - loss: 17113.7812 - val_loss: 2585.5723
Epoch 233/10000
1/1 [==============================] - 0s 15ms/step - loss: 16980.4707 - val_loss: 2653.1135
Epoch 234/10000
1/1 [==============================] - 0s 14ms/step - loss: 16863.9316 - val_loss: 2674.3789
Epoch 235/10000
1/1 [==============

1/1 [==============================] - 0s 14ms/step - loss: 13671.4590 - val_loss: 4745.2441
Epoch 302/10000
1/1 [==============================] - 0s 14ms/step - loss: 13643.7520 - val_loss: 4781.3501
Epoch 303/10000
1/1 [==============================] - 0s 14ms/step - loss: 13616.7471 - val_loss: 4817.1597
Epoch 304/10000
1/1 [==============================] - 0s 14ms/step - loss: 13590.4229 - val_loss: 4852.7061
Epoch 305/10000
1/1 [==============================] - 0s 14ms/step - loss: 13564.7559 - val_loss: 4887.9761
Epoch 306/10000
1/1 [==============================] - 0s 14ms/step - loss: 13539.7285 - val_loss: 4922.9819
Epoch 307/10000
1/1 [==============================] - 0s 14ms/step - loss: 13515.3203 - val_loss: 4957.7217
Epoch 308/10000
1/1 [==============================] - 0s 14ms/step - loss: 13491.5156 - val_loss: 4992.2080
Epoch 309/10000
1/1 [==============================] - 0s 14ms/step - loss: 13468.2969 - val_loss: 5026.4409
Epoch 310/10000
1/1 [==============

...

1/1 [==============================] - 0s 14ms/step - loss: 11934.2324 - val_loss: 9904.6729
Epoch 752/10000
1/1 [==============================] - 0s 13ms/step - loss: 11840.6289 - val_loss: 10186.0225
Epoch 753/10000
1/1 [==============================] - 0s 13ms/step - loss: 11929.9277 - val_loss: 9946.7695
Epoch 754/10000
1/1 [==============================] - 0s 14ms/step - loss: 11832.8164 - val_loss: 10161.7197
Epoch 755/10000
1/1 [==============================] - 0s 14ms/step - loss: 12113.0957 - val_loss: 10528.1777
Epoch 756/10000
1/1 [==============================] - 0s 14ms/step - loss: 11484.8496 - val_loss: 10528.6807
Epoch 757/10000
1/1 [==============================] - 0s 13ms/step - loss: 11928.2256 - val_loss: 9811.3682
Epoch 758/10000
1/1 [==============================] - 0s 14ms/step - loss: 11878.9521 - val_loss: 9293.2637
Epoch 759/10000
1/1 [==============================] - 0s 13ms/step - loss: 11924.7959 - val_loss: 10069.8037
Epoch 760/10000
1/1 [=========

1/1 [==============================] - 0s 13ms/step - loss: 11742.8672 - val_loss: 9779.1807
Epoch 827/10000
1/1 [==============================] - 0s 13ms/step - loss: 11740.2168 - val_loss: 9777.3525
Epoch 828/10000
1/1 [==============================] - 0s 13ms/step - loss: 11737.6113 - val_loss: 9775.6621
Epoch 829/10000
1/1 [==============================] - 0s 13ms/step - loss: 11735.0391 - val_loss: 9774.1191
Epoch 830/10000
1/1 [==============================] - 0s 13ms/step - loss: 11732.5049 - val_loss: 9772.6611
Epoch 831/10000
1/1 [==============================] - 0s 13ms/step - loss: 11730.0039 - val_loss: 9771.2822
Epoch 832/10000
1/1 [==============================] - 0s 13ms/step - loss: 11727.5254 - val_loss: 9770.1123
Epoch 833/10000
1/1 [==============================] - 0s 13ms/step - loss: 11725.0918 - val_loss: 9768.7803
Epoch 834/10000
1/1 [==============================] - 0s 14ms/step - loss: 11722.6787 - val_loss: 9767.6543
Epoch 835/10000
1/1 [==============

1/1 [==============================] - 0s 13ms/step - loss: 11611.6084 - val_loss: 9712.3906
Epoch 902/10000
1/1 [==============================] - 0s 13ms/step - loss: 11609.9922 - val_loss: 9711.3369
Epoch 903/10000
1/1 [==============================] - 0s 13ms/step - loss: 12473.1445 - val_loss: 8892.1299
Epoch 904/10000
1/1 [==============================] - 0s 13ms/step - loss: 11631.1699 - val_loss: 9874.8184
Epoch 905/10000
1/1 [==============================] - 0s 13ms/step - loss: 11607.9795 - val_loss: 9674.3369
Epoch 906/10000
1/1 [==============================] - 0s 13ms/step - loss: 11605.5420 - val_loss: 9712.5791
Epoch 907/10000
1/1 [==============================] - 0s 13ms/step - loss: 12403.4414 - val_loss: 2486.6829
Epoch 908/10000
1/1 [==============================] - 0s 14ms/step - loss: 16302.7246 - val_loss: 9627.5576
Epoch 909/10000
1/1 [==============================] - 0s 14ms/step - loss: 12959.5117 - val_loss: 7420.2349
Epoch 910/10000
1/1 [==============

1/1 [==============================] - 0s 14ms/step - loss: 12495.5371 - val_loss: 8911.1533
Epoch 977/10000
1/1 [==============================] - 0s 14ms/step - loss: 12493.8730 - val_loss: 8916.2295
Epoch 978/10000
1/1 [==============================] - 0s 14ms/step - loss: 12437.4111 - val_loss: 8812.6553
Epoch 979/10000
1/1 [==============================] - 0s 14ms/step - loss: 12491.7588 - val_loss: 8944.5469
Epoch 980/10000
1/1 [==============================] - 0s 14ms/step - loss: 12489.9336 - val_loss: 8919.2861
Epoch 981/10000
1/1 [==============================] - 0s 13ms/step - loss: 12488.5879 - val_loss: 8926.0850
Epoch 982/10000
1/1 [==============================] - 0s 14ms/step - loss: 12487.3359 - val_loss: 8926.6748
Epoch 983/10000
1/1 [==============================] - 0s 14ms/step - loss: 12486.1553 - val_loss: 8928.7471
Epoch 984/10000
1/1 [==============================] - 0s 14ms/step - loss: 12485.0381 - val_loss: 8930.7256
Epoch 985/10000
1/1 [==============

Epoch 1051/10000
1/1 [==============================] - 0s 13ms/step - loss: 12461.6895 - val_loss: 8999.5645
Epoch 1052/10000
1/1 [==============================] - 0s 13ms/step - loss: 12460.7520 - val_loss: 9008.3096
Epoch 1053/10000
1/1 [==============================] - 0s 13ms/step - loss: 12459.9209 - val_loss: 9015.6836
Epoch 1054/10000
1/1 [==============================] - 0s 13ms/step - loss: 12454.3291 - val_loss: 9077.4658
Epoch 1055/10000
1/1 [==============================] - 0s 13ms/step - loss: 12458.6729 - val_loss: 9017.9941
Epoch 1056/10000
1/1 [==============================] - 0s 13ms/step - loss: 12457.9814 - val_loss: 9037.4639
Epoch 1057/10000
1/1 [==============================] - 0s 13ms/step - loss: 12457.4355 - val_loss: 9041.0479
Epoch 1058/10000
1/1 [==============================] - 0s 14ms/step - loss: 12456.9707 - val_loss: 9047.1299
Epoch 1059/10000
1/1 [==============================] - 0s 13ms/step - loss: 12452.6387 - val_loss: 9094.9541
Epoch 1060

In [19]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 48ms/step
[502.824](test_target) - [[424.04346]](prediction) = 78.78054296875001


In [20]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [21]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Rondônia - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rondônia - Produção de Cimento (t),Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - Desemprego
0,-1.611543,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,1.134768,1.749525,...,0.058508,0.552610,-0.156194,-0.771315,-0.647106,-1.539414,-2.086289,-1.729822,-1.637428,1.850658
1,-1.099651,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,0.730797,1.283083,...,-0.395446,-0.192320,0.205318,-0.077198,-0.647106,-1.048104,-0.844413,-1.116076,-1.015841,1.006286
2,-0.615254,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,0.214150,0.564360,...,-1.017556,-1.088317,0.757898,0.903531,-0.647106,-0.599454,-0.498210,-0.517525,-0.541303,0.753271
3,-0.156936,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.048136,-0.180092,...,-1.322200,-1.242320,1.178995,1.537325,-0.647106,-0.191432,0.510500,-0.133020,-0.174792,-0.307113
4,0.276717,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,0.310960,-0.723717,...,-0.823757,0.238753,1.146063,1.190637,-0.620394,0.211228,0.293250,0.411988,0.266001,-0.589476
5,0.687123,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,0.778215,-0.880319,...,0.762344,1.996812,-0.086510,-0.789086,-0.070521,0.575576,0.728747,0.906803,0.630695,-0.979847
6,1.075695,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,-1.032083,-0.907852,...,1.524302,0.519469,-1.450953,-1.306218,1.111404,0.847697,1.019887,0.850652,0.768489,-0.654591
7,1.443849,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,-2.088671,-0.904988,...,1.213806,-0.784687,-1.594616,-0.687675,2.167934,1.743903,0.876528,1.327000,1.704180,-1.079188


train_target:


,Rondônia - Consumo de Cimento (t)
0,209.004
1,233.057
2,272.001
3,351.377
4,390.788
5,452.325
6,921.425
7,1021.570


test_input:


...

,Rondônia - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rondônia - Produção de Cimento (t),Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - Desemprego
17,0.923679,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,0.635756,-0.011926,...,0.437313,-0.638697,-0.57391,-0.346953,-0.082464,0.874068,-0.785213,0.698552,0.753401,-1.482458


test_target:


,Rondônia - Consumo de Cimento (t)
17,502.824


1/1 [==============================] - 0s 35ms/step
Error: 78.78054296875001




[817.366943359375,
 887.146728515625,
 831.0952758789062,
 729.5117797851562,
 794.0299072265625,
 743.3748168945312,
 700.71435546875,
 661.3301391601562,
 636.4140625,
 424.04345703125]

In [22]:
display(targets)
display(predictions)

[933.501,
 826.67,
 614.3947142857144,
 510.2737714285714,
 410.5805142857144,
 406.814,
 376.138,
 399.441,
 488.194,
 502.824]

[817.366943359375,
 887.146728515625,
 831.0952758789062,
 729.5117797851562,
 794.0299072265625,
 743.3748168945312,
 700.71435546875,
 661.3301391601562,
 636.4140625,
 424.04345703125]

In [23]:
mae = mean_absolute_error(predictions, targets)
mae

214.60256650390625

In [24]:
porcentage = mae/np.mean(targets)
porcentage

0.3924103094498737